In [13]:
import os
import sys
import random
import string
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.python.framework import ops
sys.path.append("../spykesim")
import editsim
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 80, 'linewidths':0}
ops.reset_default_graph()

In [27]:
class SNN(object):
    def __init__(self, n_in, n_hidden, n_embed, input_length):
        self.n_in = n_in
        self.n_hidden = n_hidden
        self.n_embed = n_embed
        self.input_length = input_length
        
    def bi_rnn(self, x, forget_bias = 1.0, dropout_keep_prob = 0.8):
        cell_unit = tf.contrib.rnn.BasicLSTMCell
        # Forward direction cell
        lstm_forward_cell = cell_unit(self.n_hidden,
                                      forget_bias = forget_bias)
        lstm_forward_cell = tf.contrib.rnn.DropoutWrapper(lstm_forward_cell,
                                                          output_keep_prob = dropout_keep_prob)
        # Backward direction cell
        lstm_backward_cell = cell_unit(self.n_hidden,
                                       forget_bias = forget_bias)
        lstm_backward_cell = tf.contrib.rnn.DropoutWrapper(lstm_backward_cell,
                                                           output_keep_prob = dropout_keep_prob)
        # Split title into a character sequence
        input_embed_split = tf.split(axis = 1,
                                     num_or_size_splits = self.input_length, value = x)
        input_embed_split = [tf.squeeze(input_, axis=[1]) for input_ in input_embed_split]
        outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_forward_cell,
                                                                lstm_backward_cell,
                                                                input_embed_split,
                                                                dtype=tf.float32)
        # Average The output over the sequence
        temporal_mean = tf.add_n(outputs) / input_length
        # Fully connected layer
        A = tf.get_variable(name="A", shape=[2*num_hidden, self.n_embed],
                            dtype=tf.float32,
                            initializer=tf.random_normal_initializer(stddev=0.1))
        b = tf.get_variable(name="b", shape=[self.n_embed], dtype=tf.float32,
                            initializer=tf.random_normal_initializer(stddev=0.1))        
        final_output = tf.matmul(temporal_mean, A) + b
        final_output = tf.nn.dropout(final_output, dropout_keep_prob)
        return(final_output)
    
    def inference(self, x1, x2):
        output1 = self.bi_rnn(x1)
        # Declare that we will use the same variables on the second string
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            output2 = self.bi_rnn(x2)
        # Unit normalize the outputs
        output1 = tf.nn.l2_normalize(output1, 1)
        output2 = tf.nn.l2_normalize(output2, 1)
        # Return cosine distance
        #   in this case, the dot product of the norms is the same.
        dot_prod = tf.reduce_sum(tf.multiply(output1, output2), 1)
        return(dot_prod)
    
    def loss(self, y, t):
        loss = tf.reduce_mean(tf.square(tf.subtract(scores, y_target)))
        return loss
    
    def training(self, loss):
        optimizer = tf.train.AdamOptimizer(0.01)
        train_step = optimizer.minimize(batch_loss)
        return train_step

    def fit(self, X_train, Y_train, epochs = 100, batch_size = 100):
        x1 = tf.placeholder(tf.float32, [None, nrow, ncol], name="x1")
        x2 = tf.placeholder(tf.float32, [None, nrow, ncol], name="x2")
        t = tf.placeholder(tf.float32, [None], name="target")
        dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        self._x1 = x1
        self._x2 = x2
        self._dropout_keep_prob = dropout_keep_prob
        
        y = self.inference(x1, x2)
        loss = self.loss(y, t)
        train_step = self.training(loss)
        
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        
        # For evaluate
        self._sess = sess
        
        N_train = len(X_train)
        n_batches = N_train // batch_size
        
        for epoch in range(epochs):
            X_, Y_ = shuffle(X_train, Y_train)    
            for i in range(n_batch):
                start = i * batch_size
                end = start + batch_size
                
                sess.run(train_step, feed_dict = {
                    x: X_[start:end],
                    t: Y_[start:end],
                    keep_prob: p_keep
                })
        loss_ = loss.eval(sess = sess, feed_dict = {
            x: X_train,
            t: Y_train,
            keep_prob: 1.0
        })
        self._history["loss"].append(loss_)
        return self._history

In [28]:
n_train, n_test = 1000, 100
size = 10
min_, max_ = 0, 2
nrow = 10
ncol = 10

def gen_data(min_, max_, size):
    return np.random.randint(
        min_, max_, nrow*ncol).reshape(nrow, ncol)
X_train_ = np.array([
    [gen_data(min_, max_, size), gen_data(min_, max_, size)] for _ in range(n_train)
])
X_test_ = np.array([
    [gen_data(min_, max_, size), gen_data(min_, max_, size)] for _ in range(n_test)
])
# def sim1(x, y):
#     vec = np.zeros(x.shape[1])
#     for col in range(1, x.shape[1]):
#         vec[col] = np.dot(x[:, col] + x[:, col-1],  y[:, col] + y[:, col-1])
#     return vec.mean()
Y_train_ = np.array([
    editsim.clocal_exp_editsim(xs[0].astype(np.double), xs[1].astype(np.double))[0] for xs in X_train_
])
Y_test_ = np.array([
    editsim.clocal_exp_editsim(xs[0].astype(np.double), xs[1].astype(np.double))[0] for xs in X_test_
])
xt_mean = np.mean(X_train_)
xt_std = np.std(X_train_)
yt_mean = np.mean(Y_train_)
yt_std = np.std(Y_train_)

X_train = (X_train_ - xt_mean) / xt_std
X_test =  (X_test_ - xt_mean) / xt_std
Y_train = (Y_train_ - yt_mean) / yt_std
Y_test =  (Y_test_ - yt_mean) / yt_std
# Y_train = Y_train_
# Y_test = Y_test_

In [29]:
snn = SNN(10, 10, 10, 10)

snn.fit(X_train, Y_train)

In [36]:
[x[0] for x in X_train]

[array([[-1.00081033,  0.99919033,  0.99919033, -1.00081033, -1.00081033,
         -1.00081033, -1.00081033, -1.00081033, -1.00081033,  0.99919033],
        [-1.00081033, -1.00081033, -1.00081033, -1.00081033,  0.99919033,
         -1.00081033,  0.99919033,  0.99919033,  0.99919033, -1.00081033],
        [-1.00081033,  0.99919033,  0.99919033, -1.00081033,  0.99919033,
         -1.00081033,  0.99919033,  0.99919033,  0.99919033,  0.99919033],
        [-1.00081033, -1.00081033,  0.99919033,  0.99919033, -1.00081033,
          0.99919033,  0.99919033, -1.00081033,  0.99919033, -1.00081033],
        [ 0.99919033, -1.00081033,  0.99919033,  0.99919033, -1.00081033,
         -1.00081033,  0.99919033,  0.99919033,  0.99919033,  0.99919033],
        [-1.00081033, -1.00081033,  0.99919033,  0.99919033, -1.00081033,
          0.99919033,  0.99919033,  0.99919033,  0.99919033,  0.99919033],
        [-1.00081033, -1.00081033,  0.99919033,  0.99919033, -1.00081033,
         -1.00081033,  0.9991903

In [ ]:
y